## 6장 학습 관련 기술들

### 6.1 매개변수 갱신

<img src="./img/6_1_slide.png" width="40%">

In [2]:
# import modules

import numpy as np
import matplotlib.pyplot as plt
from mnist import load_mnist
from PIL import Image
import pickle

SGD : $\eta$는 학습률(learning rate)

$\mathbf{W} \leftarrow \mathbf{W}-\eta{\partial{L}\over{\partial\mathbf{W}}}$

In [3]:
# 6.1.2 확률적 경사 하강법(SGD)

class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr

    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]

Momentum : $\alpha$는 0.9 정도로 설정함 보통

$\mathbf{v} \leftarrow \alpha\mathbf{v}-\eta{\partial{L}\over{\partial\mathbf{W}}}$

$\mathbf{W} \leftarrow \mathbf{W}+\mathbf{v}$

In [1]:
# 6.1.4 모멘텀
class Momentum:
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None

    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)

        for key in params.keys():
            self.v[key] = self.momentum * self.v[key] - self.lr * grads[key]
            params[key] += self.v[key]

AdaGrad: 개별 매개변수에 맞는 학습률을 적용하는 방법

$\mathbf{h}$에 기존 기울기 값을 계속 제곱해서 더해나가고, 학습률을 ${1\over{\sqrt{\mathbf{h}}}}$만큼 계속 조정해나간다. 이 의미는 매개변수 중 갱신이 많이 된 매개변수는 학습률이 낮아진다는 뜻이다. 

$\mathbf{h} \leftarrow \mathbf{h} + {\partial{L}\over{\partial\mathbf{W}}} \odot {\partial{L}\over{\partial\mathbf{W}}}$

$\mathbf{W} \leftarrow \mathbf{W}-\eta{1\over{\sqrt{\mathbf{h}}}}{\partial{L}\over{\partial\mathbf{W}}}$

In [4]:
# 6.1.5 AdaGrad

class AdaGrad:
    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None

    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)

        for key in params.keys():
            self.h[key] += grads[key] * grads[key]
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)

Adam : AdaGrad와 Momentum의 짬뽕

In [ ]:
""" code from https://github.com/WegraLee/deep-learning-from-scratch/blob/master/common/optimizer.py """

class Adam:

    """Adam (http://arxiv.org/abs/1412.6980v8)"""

    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.iter = 0
        self.m = None
        self.v = None
        
    def update(self, params, grads):
        if self.m is None:
            self.m, self.v = {}, {}
            for key, val in params.items():
                self.m[key] = np.zeros_like(val)
                self.v[key] = np.zeros_like(val)
        
        self.iter += 1
        lr_t  = self.lr * np.sqrt(1.0 - self.beta2**self.iter) / (1.0 - self.beta1**self.iter)         
        
        for key in params.keys():
            #self.m[key] = self.beta1*self.m[key] + (1-self.beta1)*grads[key]
            #self.v[key] = self.beta2*self.v[key] + (1-self.beta2)*(grads[key]**2)
            self.m[key] += (1 - self.beta1) * (grads[key] - self.m[key])  # 일종의 모멘텀 효과를 주는거 같다.
            self.v[key] += (1 - self.beta2) * (grads[key]**2 - self.v[key])  # 일종의 AdaGrad 효과를 주는거 같다.
            
            params[key] -= lr_t * self.m[key] / (np.sqrt(self.v[key]) + 1e-7)
            
            #unbias_m += (1 - self.beta1) * (grads[key] - self.m[key]) # correct bias
            #unbisa_b += (1 - self.beta2) * (grads[key]*grads[key] - self.v[key]) # correct bias
            #params[key] += self.lr * unbias_m / (np.sqrt(unbisa_b) + 1e-7)

### 6.2 가중치의 초깃값

sigmoid나 tanh 같은 활성화 함수의 경우에는 초기값으로 "Xavier 초기값"을 이용하는 것이 유리하다. "Xavier 초깃값"은 랜덤한 초깃값을 뽑을 때, 앞 계층의 노드가 $n$일때, 표준편차가 $\sqrt{1\over{n}}$인 정규분포를 사용한다.

ReLU를 활성화 함수로 이용할 때는 "He 초깃값"을 이용하는 것이 유리하다. "He 초깃값"은 랜덤한 초깃값을 뽑을 때, 앞 계층의 노드가 $n$일때, 표준편차가 $\sqrt{2\over{n}}$인 정규분포를 사용한다.

### 6.3 배치 정규화

각 층이 활성화를 적당히 퍼뜨릴 수 있도록 강제하는 것이 배치 정규화가 나오게 된 아이디어이다.

구체적으로는 미니배치를 단위로 데이터 분포가 평균이 0, 분산이 1이 되도록 정규화한다.

### 6.4 바른 학습을 위해

오버피팅을 피하기 위한 다양한 방법들이 제안되었다.

가중치 감소 : L1 loss, L2 loss등이 그 방법이다. 손실 함수에 가중치를 감소시키는 L1 loss, L2 loss 항을 넣어서 가중치를 전반적으로 감소시키는 방법이다.

드롭아웃 : 뉴런을 임의로 삭제하면서 학습하는 방법이다.

In [5]:
# 6.4.3 드롭아웃

class Dropout:
    def __init__(self, dropout_ratio=0.5):
        self.dropout_ratio = dropout_ratio
        self.mask = None

    def forward(self, x, train_flg=True):
        if train_flg:
            # x와 형상이 같은 배열을 무작위로 생성하고, 그 값이 dropout_ratio보다 큰 원소만 True로 설정
            self.mask = np.random.rand(*x.shape) > self.dropout_ratio
            # 여기서 순전파시 dropout에서 제외된 원소들로만 순전파가 이루어짐.
            return x * self.mask 
        else:
            return x * (1.0 - self.dropout_ratio)
    
    def backward(self, dout):
        return dout * self.mask

### 6.5 적절한 하이퍼파라미터 값 찾기

하이퍼파라미터를 시험할 때는 test data를 사용하지 말고, validation data라고 하는 새로운 데이터셋을 사용해야 한다. MNIST의 경우에는 사용자가 직접 분리해야 한다

In [ ]:
# 6.5.1 검증 데이터 분리

from mnist import load_mnist
from util import shuffle_dataset

(x_train, t_train), (x_test, t_test) = load_mnist()

x_train, t_train = shuffle_dataset(x_train, t_train)
validation_rate = 0.20
validation_num = int(x_train.shape[0] * validation_rate)

x_val = x_train[:validation_num]
t_val = t_train[:validation_num]
x_train = x_train[validation_num:]
t_train = t_train[validation_num:]

하이퍼파라미터 값을 최적화하는 과정은 다음과 같다.

- 0단계 : 하이퍼파라미터 값의 범위를 설정한다. 로그스케일로 범위를 보통 정한다.
- 1단계 : 설정된 범위에서 하이퍼파라미터의 값을 무작위로 추출합니다.
- 2단계 : 1단계에서 샘플링한 하이퍼파라미터 값을 사용하여 학습하고, 검증 데이터로 정확도를 평가한다.(에폭은 작게 설정)
- 3단계 : 1단계와 2단계를 특정 횟수(100회 등) 반복하며, 그 정확도의 결과를 보고 하이퍼파라미터의 범위를 좁힌다.

좀 감에 의존하는 것처럼 보인다. 보다 수학적으로 이를 최적화하는 방법으로는 베이즈 최적화가 있다.